In [1]:
#Paths for files
path_for_joined_geotiff_df = '' 
path_for_1km_by_1km_folder = ''
path_for_folder_to_save_models = ''

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
#from google.colab import drive
import pandas as pd
from osgeo import gdal
from skimage.transform import resize
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv(path_for_joined_geotiff_df)

In [4]:
#These Rows (from yoesmetie) had random points with elevation in the billions, so I just exluded them
outliers = [964, 970, 975, 976, 977, 982, 983, 984, 985, 992, 993, 994, 995]
df = df[~df.index.isin(outliers)]

In [5]:
batch_size = 32
steps_per_epoch = int(np.ceil(len(df)/batch_size))
num_channels = 1
num_classes = df.BIOME_NAME.nunique()
image_size = 1000
latent_dim = ((image_size*image_size)*num_classes)-num_classes
generator_input = latent_dim + num_classes

In [6]:
#Gets file name from the df and reads raster data from 1km_by_1km file
file_head = path_for_1km_by_1km_folder
dataset = []
for index, row in df.iterrows():
  path =  file_head + '/' +str(row.file_name)
  ds = gdal.Open(path)
  ds = np.array(ds.GetRasterBand(1).ReadAsArray())
  ds[ds < 0] = 0
  #Some tiles are .5 meter resolution, size 2000*2000, and some are 2 meter resolution, size 500*500.
  if np.shape(ds) != (image_size, image_size):
    #resizes all tiles to 1000*1000
    ds = resize(ds, (image_size, image_size))
  dataset.append(ds)

In [7]:
dataset = np.asarray(dataset)
#Used to normalize images
highest_point = np.max(dataset)

In [8]:
#Adds channel to data
dataset = np.expand_dims(dataset, axis=-1)

In [9]:
#Class Labels
labels =  pd.factorize(df.BIOME_NAME)
all_labels = keras.utils.to_categorical(labels[0], len(df.BIOME_NAME.unique()))

In [10]:
#Creates tesnor of raster data with class labels
dataset = tf.data.Dataset.from_tensor_slices((dataset, all_labels))
dataset = dataset.shuffle(buffer_size=len(dataset)).batch(batch_size)

In [11]:
discriminator_in_channels = num_channels + num_classes

In [12]:
def build_discriminator():
  discriminator = keras.Sequential(
      [
          keras.layers.InputLayer((image_size, image_size, discriminator_in_channels)),
          #normalize images between -1 and 1
          tf.keras.layers.Rescaling(1./highest_point, offset=-1),
          layers.BatchNormalization(momentum=0.5),
          layers.Conv2D(64, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(128, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(256, (4, 4), strides=(2, 2), padding="same" 
                        , activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(512, (4, 4), strides=(2, 2), padding="same" ,
                        activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2D(1024, (3, 3), strides=(2, 2), padding="same", 
                         activity_regularizer=keras.regularizers.L2(.00001)),
          layers.LeakyReLU(alpha=0.2),
          layers.GlobalMaxPooling2D(),
          layers.Dense(1, activation='sigmoid')
      ],
      name="discriminator",
  )
  return discriminator


def build_generator():
  generator = keras.Sequential(
      [
          keras.layers.InputLayer((generator_input,)),
          layers.Reshape((image_size, image_size, num_classes)),
          layers.Conv2DTranspose(2048, (7,7), strides=(1, 1), padding="same", activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(1024, (7, 7), strides=(1, 1), padding="same", activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(512, (7, 7), strides=(1, 1), padding="same", activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(256, (7, 7), strides=(1, 1), padding="same", activity_regularizer=keras.regularizers.L2(.00001)),
          layers.BatchNormalization(momentum=0.5),
          layers.LeakyReLU(alpha=0.2),      
          layers.Conv2DTranspose(128, (7, 7), strides=(1, 1), padding="same", activity_regularizer=keras.regularizers.L2(.00001)),
          layers.LeakyReLU(alpha=0.2),
          layers.Conv2DTranspose(1, (7, 7), strides=(1, 1), padding="same", activation='tanh'),
          layers.LeakyReLU(alpha=0.2)
      ],
      name="generator",
  )
  return generator

In [13]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        #load data
        real_images, one_hot_labels = data

        # Add dimension for the labels
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Generate random points in the latent space.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        #Add the labels
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )
        #Generate fake terrain with generator
        generated_images = self.generator(random_vector_labels)

        #Combine terrain with labels
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        #Real and Fake Terrain
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )
        
        #Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        #Track loss
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)

        return{
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [14]:
class Custom_Callback(keras.callbacks.Callback):
   def on_epoch_end(self, epoch, logs=None):
   #Saves Generator and Discriminator every 20 epochs
         if (epoch%20 == 0) and (epoch != 0):
           cond_gan.generator.save(path_for_folder_to_save_models + '/generator_' + str(epoch))
           cond_gan.discriminator.save(path_for_folder_to_save_models + '/discriminator_' + str(epoch))
       #If discriminator loss is very close to zero training stops  
         if logs.get('d_loss') < .0001:
           self.model.stop_training = True
custom_checkpoint = Custom_Callback()

In [15]:
#Learning rate decays at rate initial_learning_rate * .9986^(decay_steps/steps_per_epoch)
g_initial_learning_rate = 0.001
d_initial_learning_rate = 0.001
g_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    g_initial_learning_rate,
    decay_steps=steps_per_epoch,
    decay_rate=.9986)
d_lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    d_initial_learning_rate,
    decay_steps=steps_per_epoch,
    decay_rate=.9986)

In [16]:
generator = build_generator()
discriminator = build_discriminator()

cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=d_lr_schedule),
    g_optimizer=keras.optimizers.Adam(learning_rate=g_lr_schedule),
    loss_fn=keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM),
)

In [17]:
cond_gan.fit(dataset, epochs=700, callbacks=[custom_checkpoint])

Epoch 1/700


ResourceExhaustedError: ignored

In [ ]:
cond_gan.generator.save(path_for_folder_to_save_models + "/generator_final")
cond_gan.discriminator.save(path_for_folder_to_save_models + "/discriminator_final")